# sodapy tutorial using NYC Open Data 
Mark Bauer

# Introduction  
This notebook demonstrates how to use sodapy, the python client for the Socrata Open Data API, with NYC Open Data. Examples of popular methods are included, as well as basic queries using SoQL, the Socrata Query Language. 

# sodapy
sodapy is a python client for the Socrata Open Data API.

# Information about sodapy
 
**Installing**: https://pypi.org/project/sodapy/  
**GitHub**: https://github.com/xmunoz/sodapy

**The official Socrata Open Data API (SODA) docs**  
https://dev.socrata.com/

**Queries using SODA**  
https://dev.socrata.com/docs/queries/

**Inspiration for this notebook**  
https://github.com/xmunoz/sodapy/blob/master/examples/soql_queries.ipynb

# Importing Libraries

In [152]:
# importing libraries
import pandas as pd
from sodapy import Socrata
import itertools 

In [153]:
%reload_ext watermark

In [154]:
%watermark -a "Mark Bauer" -u -t -d -v -p pandas,sodapy,itertools

Mark Bauer 
last updated: 2021-01-28 12:49:02 

CPython 3.7.1
IPython 7.18.1

pandas 1.0.0
sodapy 2.0.0
itertools unknown


Documention for installing watermark: https://github.com/rasbt/watermark

# Using sodapy

In order use sodapy, a source domain (i.e. the open data source you are trying to connect to) needs to be passed to the Socrata class. Additionally, if a user wants to query a specific data set, then the data set identifier (i.e. the data set id on the given source domain) needs to be passed as well. Below, we identify NYC Open Data's source domain: `data.cityofnewyork.us` and the data set identifier for the NYC 311 data set: `erm2-nwe9`. The screenshot is the homepage of the 311 data set from NYC Open Data.

![nyc-311-api-docs](nyc-311-api-docs.png)  

Source: https://dev.socrata.com/foundry/data.cityofnewyork.us/erm2-nwe9

We assign this information to `socrata_domain` and `socrata_dataset_identifier` variables below.

In [155]:
socrata_domain = 'data.cityofnewyork.us'
socrata_dataset_identifier = 'erm2-nwe9'

# Socrata class in sodapy

In [156]:
# The main class that interacts with the SODA API.

# The required arguments are:
#     domain: the domain you wish you to access
#     app_token: your Socrata application token
# Simple requests are possible without an app_token, though these
# requests will be rate-limited.

client = Socrata(socrata_domain, None, timeout=1000)

In [157]:
type(client)

sodapy.Socrata

Socrata Methods

![socrata-methods](socrata-methods.png)

Source: https://github.com/xmunoz/sodapy#datasetslimit0-offset0

We will review a few of the popular methods in this tutorial.

# Socrata Methods

## `.datasets()`

`datasets` method: Returns the list of datasets associated with a particular domain.
WARNING: Large limits (>1000) will return megabytes of data,
which can be slow on low-bandwidth networks, and is also a lot of
data to hold in memory.

In [158]:
print(type(client.datasets()))

<class 'list'>


In [159]:
print(type(client.datasets()[0]))

<class 'dict'>


In [160]:
client.datasets()[0].keys()

dict_keys(['resource', 'classification', 'metadata', 'permalink', 'link', 'owner', 'creator'])

In [161]:
# Viewing the resource dictionary in the first item of the datasets list. From there, 
# we view the first five items under the resource dictionary.

limit = 5
dict(itertools.islice(client.datasets()[0]['resource'].items(), limit))

{'name': 'DOB Job Application Filings',
 'id': 'ic3t-wcy2',
 'parent_fxf': [],
 'description': 'This dataset contains all job applications submitted through the Borough Offices, through eFiling, or through the HUB, which have a "Latest Action Date" since January 1, 2000. This dataset does not include jobs submitted through DOB NOW. See the DOB NOW: Build – Job Application Filings dataset for DOB NOW jobs.',
 'attribution': 'Department of Buildings (DOB)'}

In [162]:
# Once we've identified the structure of the dictionary, we try to 
# find the 311 data set and identify its position in the datasets list.

idx = 0

for dataset_id in client.datasets():
    if client.datasets()[idx]['resource']['id'] == 'erm2-nwe9':
        print(client.datasets()[idx]['resource']['name'], \
              '\nindex:', idx)
        break
    else:
        idx += 1    

311 Service Requests from 2010 to Present 
index: 5


In [163]:
# Previewing information about the 311 data set from the datasets method.
# Note: Information is quite long.

idx_311 = idx
print('The 311 data set index in the datasets() list is:', idx_311)


print('Previewing information about the 311 data set.')
client.datasets()[idx_311]

The 311 data set index in the datasets() list is: 5
Previewing information about the 311 data set.


{'resource': {'name': '311 Service Requests from 2010 to Present',
  'id': 'erm2-nwe9',
  'parent_fxf': [],
  'description': '<b>NOTE: This data does not present a full picture of 311 calls or service requests, in part because of operational and system complexities associated with remote call taking necessitated by the unprecedented volume 311 is handling during the Covid-19 crisis. The City is working to address this issue. </b>\r\n\r\nAll 311 Service Requests from 2010 to present. This information is automatically updated daily.',
  'attribution': '311, DoITT',
  'attribution_link': None,
  'contact_email': None,
  'type': 'dataset',
  'updatedAt': '2021-01-28T02:41:32.000Z',
  'createdAt': '2011-10-10T05:52:17.000Z',
  'metadata_updated_at': '2020-04-22T20:18:38.000Z',
  'data_updated_at': '2021-01-28T02:41:32.000Z',
  'page_views': {'page_views_last_week': 1435,
   'page_views_last_month': 5157,
   'page_views_total': 438773,
   'page_views_last_week_log': 10.487840033823051,
   'p

In [164]:
# Since the datasets method is long, let's see if we can identify specific keys we want to preview
# in the resource dictionary.

for key in client.datasets()[idx_311]['resource'].keys():
    print(key)

name
id
parent_fxf
description
attribution
attribution_link
contact_email
type
updatedAt
createdAt
metadata_updated_at
data_updated_at
page_views
columns_name
columns_field_name
columns_datatype
columns_description
columns_format
download_count
provenance
lens_view_type
blob_mime_type
hide_from_data_json
publication_date


In [165]:
items = client.datasets()[idx_311]['resource'].items()

for key, value in items:
    print(key + ':', str(value) + '\n')

name: 311 Service Requests from 2010 to Present

id: erm2-nwe9

parent_fxf: []

description: <b>NOTE: This data does not present a full picture of 311 calls or service requests, in part because of operational and system complexities associated with remote call taking necessitated by the unprecedented volume 311 is handling during the Covid-19 crisis. The City is working to address this issue. </b>

All 311 Service Requests from 2010 to present. This information is automatically updated daily.

attribution: 311, DoITT

attribution_link: None

contact_email: None

type: dataset

updatedAt: 2021-01-28T02:41:32.000Z

createdAt: 2011-10-10T05:52:17.000Z

metadata_updated_at: 2020-04-22T20:18:38.000Z

data_updated_at: 2021-01-28T02:41:32.000Z

page_views: {'page_views_last_week': 1435, 'page_views_last_month': 5157, 'page_views_total': 438773, 'page_views_last_week_log': 10.487840033823051, 'page_views_last_month_log': 12.332596057789216, 'page_views_total_log': 18.743118514347493}

columns_

## `.get()`

`get` method: Read data from the requested resource. Options for content_type are json,
csv, and xml.

In [166]:
client = Socrata(socrata_domain, None, timeout=100)

In [167]:
# Using try and except statements because these requests are large and may timeout.
# If the request timesout, we skip it. 

try:
    print(type(client.get(socrata_dataset_identifier)))
except:
    print('timeout error. skipping.')
    pass  

<class 'list'>


In [168]:
# Using try and except statements because these requests are large and may timeout.
# If the request timesout, we skip it. 

# printing the column headers

keys = client.get(socrata_dataset_identifier, select='*')[0].keys()

try:
    for key in keys:
        print(key)
except:
    print('timeout error. skipping.')
    pass

unique_key
created_date
agency
agency_name
complaint_type
descriptor
location_type
incident_zip
incident_address
street_name
cross_street_1
cross_street_2
intersection_street_1
intersection_street_2
city
landmark
status
community_board
bbl
borough
x_coordinate_state_plane
y_coordinate_state_plane
open_data_channel_type
park_facility_name
park_borough
latitude
longitude
location
:@computed_region_efsh_h5xi
:@computed_region_f5dn_yrer
:@computed_region_yeji_bk3q
:@computed_region_92fq_4b7q
:@computed_region_sbqj_enih


In [169]:
# Printing the column and value of the first record

items = client.get(socrata_dataset_identifier, select='*')[0].items()

try:
    for key, value in items:
        print(key + ':',  value)
except:
    print('timeout error. skipping.')
    pass        

unique_key: 49633221
created_date: 2021-01-27T01:59:12.000
agency: DOHMH
agency_name: Department of Health and Mental Hygiene
complaint_type: Rodent
descriptor: Rat Sighting
location_type: 1-2 Family Dwelling
incident_zip: 11238
incident_address: 465 SAINT JOHN'S PLACE
street_name: SAINT JOHN'S PLACE
cross_street_1: WASHINGTON AVENUE
cross_street_2: CLASSON AVENUE
intersection_street_1: WASHINGTON AVENUE
intersection_street_2: CLASSON AVENUE
city: BROOKLYN
landmark: ST JOHNS PLACE
status: In Progress
community_board: 08 BROOKLYN
bbl: 3011740061
borough: BROOKLYN
x_coordinate_state_plane: 994697
y_coordinate_state_plane: 184641
open_data_channel_type: PHONE
park_facility_name: Unspecified
park_borough: BROOKLYN
latitude: 40.6734674790127
longitude: -73.9623383832787
location: {'latitude': '40.6734674790127', 'longitude': '-73.9623383832787', 'human_address': '{"address": "", "city": "", "state": "", "zip": ""}'}
:@computed_region_efsh_h5xi: 13829
:@computed_region_f5dn_yrer: 16
:@comput

## `.get_metadata()`

`get_metadata` method: Retrieve the metadata for a particular dataset.

In [170]:
client = Socrata(socrata_domain, None, timeout=1000)

In [171]:
type(client.get_metadata(socrata_dataset_identifier))

dict

In [172]:
# Previewing keys vertically.
keys = client.get_metadata(socrata_dataset_identifier).keys()
for key in keys:
    print(key)

id
name
assetType
attribution
averageRating
category
createdAt
description
displayType
downloadCount
hideFromCatalog
hideFromDataJson
indexUpdatedAt
newBackend
numberOfComments
oid
provenance
publicationAppendEnabled
publicationDate
publicationGroup
publicationStage
rowClass
rowIdentifierColumnId
rowsUpdatedAt
rowsUpdatedBy
tableId
totalTimesRated
viewCount
viewLastModified
viewType
approvals
columns
grants
metadata
owner
query
rights
tableAuthor
tags
flags


In [173]:
# Previewing the id and name of the data set.
print('id and name of dataset\n' + \
      '-' * 30 + \
      '\nid:', client.get_metadata(socrata_dataset_identifier)['id'], \
      '\nname:', client.get_metadata(socrata_dataset_identifier)['name'])

id and name of dataset
------------------------------
id: erm2-nwe9 
name: 311 Service Requests from 2010 to Present


In [174]:
# Previewing the first 30 keys, values of the dictionary.
metadata = client.get_metadata(socrata_dataset_identifier)

limit = 30
out = dict(itertools.islice(metadata.items(), limit))
print(type(out), '\n')

for key, value in out.items():
    print(key + ':',  value)

<class 'dict'> 

id: erm2-nwe9
name: 311 Service Requests from 2010 to Present
assetType: dataset
attribution: 311, DoITT
averageRating: 0
category: Social Services
createdAt: 1318225937
description: <b>NOTE: This data does not present a full picture of 311 calls or service requests, in part because of operational and system complexities associated with remote call taking necessitated by the unprecedented volume 311 is handling during the Covid-19 crisis. The City is working to address this issue. </b>

All 311 Service Requests from 2010 to present. This information is automatically updated daily.
displayType: table
downloadCount: 398304
hideFromCatalog: False
hideFromDataJson: False
indexUpdatedAt: 1571326778
newBackend: True
numberOfComments: 19
oid: 28506835
provenance: official
publicationAppendEnabled: False
publicationDate: 1524193398
publicationGroup: 244403
publicationStage: published
rowClass: 
rowIdentifierColumnId: 354922030
rowsUpdatedAt: 1611801692
rowsUpdatedBy: 5fuc-pqz2

In [175]:
# Saving metadata dictionary as 'metadata'
metadata = client.get_metadata(socrata_dataset_identifier)
print(type(metadata))

# Previewing the datatype of columns
print(type(metadata['columns']), ', length:', len(metadata['columns']))

# Previewing the field names for each element in our columns list
for x in metadata['columns']:
    print(x['fieldName'])

<class 'dict'>
<class 'list'> , length: 46
unique_key
created_date
closed_date
agency
agency_name
complaint_type
descriptor
location_type
incident_zip
incident_address
street_name
cross_street_1
cross_street_2
intersection_street_1
intersection_street_2
address_type
city
landmark
facility_type
status
due_date
resolution_description
resolution_action_updated_date
community_board
bbl
borough
x_coordinate_state_plane
y_coordinate_state_plane
open_data_channel_type
park_facility_name
park_borough
vehicle_type
taxi_company_borough
taxi_pick_up_location
bridge_highway_name
bridge_highway_direction
road_ramp
bridge_highway_segment
latitude
longitude
location
:@computed_region_efsh_h5xi
:@computed_region_f5dn_yrer
:@computed_region_yeji_bk3q
:@computed_region_92fq_4b7q
:@computed_region_sbqj_enih


In [176]:
metadata = client.get_metadata(socrata_dataset_identifier)

# Previewing the first element in our columns list
metadata['columns'][0]

{'id': 354922030,
 'name': 'Unique Key',
 'dataTypeName': 'text',
 'description': 'Unique identifier of a Service Request (SR) in the open data set\n',
 'fieldName': 'unique_key',
 'position': 1,
 'renderTypeName': 'text',
 'tableColumnId': 1567787,
 'width': 220,
 'cachedContents': {'largest': '49633815',
  'non_null': '24816638',
  'null': '0',
  'top': [{'item': '10693408', 'count': '1'},
   {'item': '10836749', 'count': '1'},
   {'item': '10836967', 'count': '1'},
   {'item': '11051177', 'count': '1'},
   {'item': '11413576', 'count': '1'},
   {'item': '11463895', 'count': '1'},
   {'item': '11463896', 'count': '1'},
   {'item': '11464334', 'count': '1'},
   {'item': '11464394', 'count': '1'},
   {'item': '11464467', 'count': '1'},
   {'item': '11464508', 'count': '1'},
   {'item': '11464509', 'count': '1'},
   {'item': '11464521', 'count': '1'},
   {'item': '11464567', 'count': '1'},
   {'item': '11464572', 'count': '1'},
   {'item': '11464639', 'count': '1'},
   {'item': '1146484

In [177]:
metadata = client.get_metadata(socrata_dataset_identifier)

# Identifying our columns list
cols = metadata['columns']

# Previewing first position in our columns list
cols[1]

{'id': 354922031,
 'name': 'Created Date',
 'dataTypeName': 'calendar_date',
 'description': 'Date SR  was created\n',
 'fieldName': 'created_date',
 'position': 2,
 'renderTypeName': 'calendar_date',
 'tableColumnId': 1567788,
 'width': 244,
 'cachedContents': {'largest': '2021-01-27T01:59:12.000',
  'non_null': '24816638',
  'null': '0',
  'top': [{'item': '2013-01-24T00:00:00.000', 'count': '7650'},
   {'item': '2015-01-08T00:00:00.000', 'count': '7242'},
   {'item': '2014-01-07T00:00:00.000', 'count': '7030'},
   {'item': '2015-02-16T00:00:00.000', 'count': '6430'},
   {'item': '2014-01-08T00:00:00.000', 'count': '6197'},
   {'item': '2012-01-04T00:00:00.000', 'count': '5933'},
   {'item': '2013-11-25T00:00:00.000', 'count': '5909'},
   {'item': '2014-01-23T00:00:00.000', 'count': '5782'},
   {'item': '2014-01-22T00:00:00.000', 'count': '5497'},
   {'item': '2015-01-07T00:00:00.000', 'count': '5432'},
   {'item': '2011-01-24T00:00:00.000', 'count': '5380'},
   {'item': '2011-10-28T

In [178]:
# Creating a fieldName dictionary for every element in our column list
fieldName = {x['fieldName']: x for x in cols}

# Previewing the field names (values) in our fieldName dictionary
for key in fieldName.keys():
    print(key)

unique_key
created_date
closed_date
agency
agency_name
complaint_type
descriptor
location_type
incident_zip
incident_address
street_name
cross_street_1
cross_street_2
intersection_street_1
intersection_street_2
address_type
city
landmark
facility_type
status
due_date
resolution_description
resolution_action_updated_date
community_board
bbl
borough
x_coordinate_state_plane
y_coordinate_state_plane
open_data_channel_type
park_facility_name
park_borough
vehicle_type
taxi_company_borough
taxi_pick_up_location
bridge_highway_name
bridge_highway_direction
road_ramp
bridge_highway_segment
latitude
longitude
location
:@computed_region_efsh_h5xi
:@computed_region_f5dn_yrer
:@computed_region_yeji_bk3q
:@computed_region_92fq_4b7q
:@computed_region_sbqj_enih


In [179]:
# Removing the last five field names
list(fieldName.keys())[:-5]

['unique_key',
 'created_date',
 'closed_date',
 'agency',
 'agency_name',
 'complaint_type',
 'descriptor',
 'location_type',
 'incident_zip',
 'incident_address',
 'street_name',
 'cross_street_1',
 'cross_street_2',
 'intersection_street_1',
 'intersection_street_2',
 'address_type',
 'city',
 'landmark',
 'facility_type',
 'status',
 'due_date',
 'resolution_description',
 'resolution_action_updated_date',
 'community_board',
 'bbl',
 'borough',
 'x_coordinate_state_plane',
 'y_coordinate_state_plane',
 'open_data_channel_type',
 'park_facility_name',
 'park_borough',
 'vehicle_type',
 'taxi_company_borough',
 'taxi_pick_up_location',
 'bridge_highway_name',
 'bridge_highway_direction',
 'road_ramp',
 'bridge_highway_segment',
 'latitude',
 'longitude',
 'location']

In [180]:
# Removing the last five field names
cols_as_list = list(fieldName.keys())[:-5]

cols_as_list

['unique_key',
 'created_date',
 'closed_date',
 'agency',
 'agency_name',
 'complaint_type',
 'descriptor',
 'location_type',
 'incident_zip',
 'incident_address',
 'street_name',
 'cross_street_1',
 'cross_street_2',
 'intersection_street_1',
 'intersection_street_2',
 'address_type',
 'city',
 'landmark',
 'facility_type',
 'status',
 'due_date',
 'resolution_description',
 'resolution_action_updated_date',
 'community_board',
 'bbl',
 'borough',
 'x_coordinate_state_plane',
 'y_coordinate_state_plane',
 'open_data_channel_type',
 'park_facility_name',
 'park_borough',
 'vehicle_type',
 'taxi_company_borough',
 'taxi_pick_up_location',
 'bridge_highway_name',
 'bridge_highway_direction',
 'road_ramp',
 'bridge_highway_segment',
 'latitude',
 'longitude',
 'location']

# Socrata Query Language (SoQL) - Analyzing NYC 311 Complaints

## Most NYC 311 Complaints by Complaint Type

In [181]:
# Source domain for NYC Open Data on Socrata
socrata_domain = 'data.cityofnewyork.us'

# Data set id for NYC 311 on NYC Open Data on Socrata
socrata_dataset_identifier = 'erm2-nwe9'

# Socrata - The main class that interacts with the SODA API. We pass the source domain value
# of NYC Open data, the app token as 'None', and set the timeout parameter for '1,000 seconds'
client = Socrata(socrata_domain, None, timeout=1000)

# SoQL query string below:
# Select the complaint_type and the count of complaint_type columns 
# grouped by complaint_type, sorted by the count of complaint_type in descending order
# and limit our records to 1,000.

query = """
SELECT 
    complaint_type, 
    count(complaint_type) AS count
GROUP BY 
    complaint_type
ORDER BY 
    count(complaint_type) DESC
LIMIT 
    1000
"""

# Requesting data from the NYC 311 data set
# and passing our query as a full SoQL query string
results = client.get(socrata_dataset_identifier, query=query)

# results is returned as JSON from API and converted to Python list of
# dictionaries by sodapy
print(type(results), 'Returned a list from our request.\n')

# Identifying type of first element of our results list
print(type(results[0]), 'However, request is actually a list of dictionaries.\n')

# Convert list of dictionaries to a pandas DataFrame
results_df = pd.DataFrame.from_records(results)

print(type(results_df), 'Convert list of dictionaries to DataFrame.')
print('Rows and columns of data:', results_df.shape)

results_df.head(10)

<class 'list'> Returned a list from our request.

<class 'dict'> However, request is actually a list of dictionaries.

<class 'pandas.core.frame.DataFrame'> Convert list of dictionaries to DataFrame.
Rows and columns of data: (445, 2)


,complaint_type,count
0,Noise - Residential,2238037
1,HEAT/HOT WATER,1423237
2,Illegal Parking,1126019
3,Blocked Driveway,1045927
4,Street Condition,1020888
5,Street Light Condition,984066
6,HEATING,887869
7,PLUMBING,747937
8,Water System,687400
9,Noise - Street/Sidewalk,682532


## Most NYC 311 Complaints by Descriptor

In [182]:
# Source domain for NYC Open Data on Socrata
socrata_domain = 'data.cityofnewyork.us'

# Data set id for NYC 311 on NYC Open Data on Socrata
socrata_dataset_identifier = 'erm2-nwe9'

# Socrata - The main class that interacts with the SODA API. We pass the source domain value
# of NYC Open data, the app token as 'None', and set the timeout parameter for '1,000 seconds'
client = Socrata(socrata_domain, None, timeout=1000)

# SoQL query string below:
# Select the descriptor and count of descriptor columns grouped by descriptor,
# sort by the count in descending order and limit our records to 1,000.

query = """
SELECT 
    descriptor, 
    count(descriptor) AS count
GROUP BY 
    descriptor
ORDER BY 
    count DESC
LIMIT 
    1000
"""

# Requesting data from the NYC 311 data set
# and passing our query as a full SoQL query string
results = client.get(socrata_dataset_identifier, query=query)

# Convert list of dictionaries to a pandas DataFrame
results_df = pd.DataFrame.from_records(results)

print('Rows and columns of data:', results_df.shape)

results_df.head(10)

Rows and columns of data: (1000, 2)


,descriptor,count
0,Loud Music/Party,2336486
1,ENTIRE BUILDING,926711
2,HEAT,871935
3,No Access,778808
4,Street Light Out,729603
5,Pothole,617865
6,Banging/Pounding,613329
7,APARTMENT ONLY,496526
8,Loud Talking,359034
9,CEILING,358956


## Most NYC 311 Complaints by Day

In [183]:
# Source domain for NYC Open Data on Socrata
socrata_domain = 'data.cityofnewyork.us'

# Data set id for NYC 311 on NYC Open Data on Socrata
socrata_dataset_identifier = 'erm2-nwe9'

# Socrata - The main class that interacts with the SODA API. We pass the source domain value
# of NYC Open data, the app token as 'None', and set the timeout parameter for '1,000 seconds'
client = Socrata(socrata_domain, None, timeout=1000)

# SoQL query string below:
# Select the day and count day grouped by day,
# sort by count in descending order and limit our records to 1,000.

query = """
SELECT 
    date_trunc_ymd(created_date) AS day, 
    count(day) AS count
GROUP BY 
    day
ORDER BY 
    count DESC
LIMIT 
    1000
"""

# Requesting data from the NYC 311 data set
# and passing our query as a full SoQL query string
results = client.get(socrata_dataset_identifier, query=query)

# Convert list of dictionaries to a pandas DataFrame
results_df = pd.DataFrame.from_records(results)

print('Rows and columns of data:', results_df.shape)

results_df.head(10)

Rows and columns of data: (1000, 2)


,day,count
0,2020-08-04T00:00:00.000,23314
1,2020-08-05T00:00:00.000,18305
2,2020-07-05T00:00:00.000,16014
3,2020-07-04T00:00:00.000,15365
4,2020-06-20T00:00:00.000,15098
5,2020-06-21T00:00:00.000,14965
6,2020-06-28T00:00:00.000,12899
7,2020-06-27T00:00:00.000,12074
8,2020-08-09T00:00:00.000,12057
9,2020-08-06T00:00:00.000,12043


## Displaying the difference between the date (timestamp) and day (date_trunc_ymd) columns

In [184]:
# Source domain for NYC Open Data on Socrata
socrata_domain = 'data.cityofnewyork.us'

# Data set id for NYC 311 on NYC Open Data on Socrata
socrata_dataset_identifier = 'erm2-nwe9'

# Socrata - The main class that interacts with the SODA API. We pass the source domain value
# of NYC Open data, the app token as 'None', and set the timeout parameter for '1,000 seconds'
client = Socrata(socrata_domain, None, timeout=1000)

# SoQL query string below:
# Select the timestamp, day and count timestamp grouped by timestamp,
# sort by count in descending order and limit our records to 1,000.

query = """
SELECT 
    created_date as timestamp, 
    date_trunc_ymd(created_date) as day, 
    count(timestamp) AS count
GROUP BY 
    timestamp
ORDER BY 
    count ASC
LIMIT 
    1000
"""

# Requesting data from the NYC 311 data set
# and passing our query as a full SoQL query string
results = client.get(socrata_dataset_identifier, query=query)

# Convert list of dictionaries to a pandas DataFrame
results_df = pd.DataFrame.from_records(results)

print('Rows and columns of data:', results_df.shape)

results_df.head(10)

Rows and columns of data: (1000, 3)


,timestamp,day,count
0,2010-01-01T15:48:17.000,2010-01-01T00:00:00.000,1
1,2010-01-01T16:01:57.000,2010-01-01T00:00:00.000,1
2,2010-01-01T15:40:55.000,2010-01-01T00:00:00.000,1
3,2010-01-01T15:48:01.000,2010-01-01T00:00:00.000,1
4,2010-01-01T15:57:07.000,2010-01-01T00:00:00.000,1
5,2010-01-01T16:01:43.000,2010-01-01T00:00:00.000,1
6,2010-01-01T15:35:00.000,2010-01-01T00:00:00.000,1
7,2010-01-01T15:39:32.000,2010-01-01T00:00:00.000,1
8,2010-01-01T15:45:00.000,2010-01-01T00:00:00.000,1
9,2010-01-01T15:48:00.000,2010-01-01T00:00:00.000,1


## Analyzing NYC 311 Street Flooding Complaints

### Searching the data set for the word "flood" in the complaint_type field

In [185]:
# Source domain for NYC Open Data on Socrata
socrata_domain = 'data.cityofnewyork.us'

# Data set id for NYC 311 on NYC Open Data on Socrata
socrata_dataset_identifier = 'erm2-nwe9'

# Socrata - The main class that interacts with the SODA API. We pass the source domain value
# of NYC Open data, the app token as 'None', and set the timeout parameter for '1,000 seconds'
client = Socrata(socrata_domain, None, timeout=1000)

# SoQL query string below:
# Select complaint_type and count of complaint_type grouped by compaint_type,
# where the word "flood" is in compplaint_type,
# sort count in descending order and limit our records to 1,000.

query = """
SELECT 
    complaint_type, 
    count(complaint_type) AS count
WHERE 
    LOWER(complaint_type) LIKE '%flood%'
GROUP BY 
    complaint_type
ORDER BY 
    count(descriptor) DESC
LIMIT 
    1000
"""

# Returned as JSON from API / converted to Python list of
# dictionaries by sodapy.
results = client.get(socrata_dataset_identifier, query=query)

# Convert to pandas DataFrame
results_df = pd.DataFrame.from_records(results)

print('shape of data:', results_df.shape)
results_df

shape of data: (0, 0)


""


### Searching the data set for the word "flood" in the descriptor field

In [186]:
# Source domain for NYC Open Data on Socrata
socrata_domain = 'data.cityofnewyork.us'

# Data set id for NYC 311 on NYC Open Data on Socrata
socrata_dataset_identifier = 'erm2-nwe9'

# Socrata - The main class that interacts with the SODA API. We pass the source domain value
# of NYC Open data, the app token as 'None', and set the timeout parameter for '1,000 seconds'
client = Socrata(socrata_domain, None, timeout=1000)

# SoQL query string below:
# Select complaint_type and count of complaint_type grouped by compaint_type,
# where the word "flood" is in descriptor,
# sort count in descending order and limit our records to 1,000.

query = """
SELECT 
    complaint_type, 
    count(complaint_type) AS count
WHERE 
    LOWER(descriptor) LIKE '%flood%'
GROUP BY 
    complaint_type
ORDER BY 
    count(descriptor) DESC
LIMIT 
    1000
"""

# Returned as JSON from API / converted to Python list of
# dictionaries by sodapy.
results = client.get(socrata_dataset_identifier, query=query)

# Convert to pandas DataFrame
results_df = pd.DataFrame.from_records(results)

print('shape of data:', results_df.shape)
results_df

shape of data: (4, 2)


,complaint_type,count
0,Sewer,120281
1,Street Light Condition,9047
2,OEM Literature Request,271
3,Public Toilet,48


### Searching the data set where complaint_type field = 'Sewer'

In [187]:
# Source domain for NYC Open Data on Socrata
socrata_domain = 'data.cityofnewyork.us'

# Data set id for NYC 311 on NYC Open Data on Socrata
socrata_dataset_identifier = 'erm2-nwe9'

# Socrata - The main class that interacts with the SODA API. We pass the source domain value
# of NYC Open data, the app token as 'None', and set the timeout parameter for '1,000 seconds'
client = Socrata(socrata_domain, None, timeout=1000)

# SoQL query string below:
# Select descriptor and count of descriptor grouped by descriptor,
# where complaint_type = 'Sewer',
# sort count in descending order and limit our records to 1,000.

query = """
SELECT 
    descriptor, 
    count(descriptor)
WHERE 
    complaint_type='Sewer'
GROUP BY 
    descriptor
ORDER BY 
    count(descriptor) DESC
LIMIT 1000
"""

# First 1000 results, returned as JSON from API / converted to Python list of
# dictionaries by sodapy.
results = client.get(socrata_dataset_identifier, query=query)

# Convert to pandas DataFrame
results_df = pd.DataFrame.from_records(results)

print('shape of data:', results_df.shape)
results_df.head(len(results_df))

shape of data: (27, 2)


,descriptor,count_descriptor
0,Sewer Backup (Use Comments) (SA),148695
1,Catch Basin Clogged/Flooding (Use Comments) (SC),89849
2,Catch Basin Sunken/Damaged/Raised (SC1),28528
3,Street Flooding (SJ),27520
4,Manhole Cover Broken/Making Noise (SB),19778
5,Manhole Cover Missing (Emergency) (SA3),17430
6,Sewer Odor (SA2),15351
7,Defective/Missing Curb Piece (SC4),8486
8,Manhole Overflow (Use Comments) (SA1),6842
9,Catch Basin Search (SC2),4153


### Searching the data set where the word "flood" is in the descriptor field

In [188]:
# Source domain for NYC Open Data on Socrata
socrata_domain = 'data.cityofnewyork.us'

# Data set id for NYC 311 on NYC Open Data on Socrata
socrata_dataset_identifier = 'erm2-nwe9'

# Socrata - The main class that interacts with the SODA API. We pass the source domain value
# of NYC Open data, the app token as 'None', and set the timeout parameter for '1,000 seconds'
client = Socrata(socrata_domain, None, timeout=1000)

# SoQL query string below:
# Select descriptor and count of descriptor grouped by descriptor,
# where the word "flood" is in descriptor,
# sort count in descending order and limit our records to 1,000.

query = """
SELECT 
    descriptor, 
    count(descriptor) AS count
WHERE 
    LOWER(descriptor) LIKE '%flood%'
GROUP BY 
    descriptor
ORDER BY 
    count(descriptor) DESC
LIMIT 
    1000
"""

# Returned as JSON from API / converted to Python list of
# dictionaries by sodapy.
results = client.get(socrata_dataset_identifier, query=query)

# Convert to pandas DataFrame
results_df = pd.DataFrame.from_records(results)

print('shape of data:', results_df.shape)
results_df

shape of data: (11, 2)


,descriptor,count
0,Catch Basin Clogged/Flooding (Use Comments) (SC),89849
1,Street Flooding (SJ),27520
2,Flood Light Lamp Out,5964
3,Highway Flooding (SH),2834
4,Flood Light Lamp Cycling,2511
5,Ready NY - Flooding,271
6,Flood Light Lamp Dayburning,205
7,Flood Light Lamp Missing,190
8,Flood Light Lamp Dim,177
9,RAIN GARDEN FLOODING (SRGFLD),78


### Displaying the highest number of street flooding complaints by day

In [189]:
# Source domain for NYC Open Data on Socrata
socrata_domain = 'data.cityofnewyork.us'

# Data set id for NYC 311 on NYC Open Data on Socrata
socrata_dataset_identifier = 'erm2-nwe9'

# Socrata - The main class that interacts with the SODA API. We pass the source domain value
# of NYC Open data, the app token as 'None', and set the timeout parameter for '1,000 seconds'
client = Socrata(socrata_domain, None, timeout=1000)

# SoQL query string below:
# Select the day and the count day columns grouped by day,
# where the descriptor is Street Flooding (SJ),
# sort the count in descending order and limit our records to 1,000.

query = """
SELECT 
    date_trunc_ymd(created_date) as day, 
    count(created_date) AS count
WHERE 
    descriptor == 'Street Flooding (SJ)'
GROUP BY 
    day
ORDER BY 
    count DESC
LIMIT 
    1000
"""

# Returned as JSON from API / converted to Python list of
# dictionaries by sodapy.
results = client.get(socrata_dataset_identifier, query=query)

# Convert to pandas DataFrame
results_df = pd.DataFrame.from_records(results)

print('shape of data:', results_df.shape)

results_df.head(10)

shape of data: (1000, 2)


,day,count
0,2017-05-05T00:00:00.000,247
1,2014-12-09T00:00:00.000,226
2,2014-04-30T00:00:00.000,189
3,2018-04-16T00:00:00.000,163
4,2013-05-08T00:00:00.000,162
5,2016-11-15T00:00:00.000,151
6,2016-02-08T00:00:00.000,150
7,2018-11-25T00:00:00.000,142
8,2020-07-10T00:00:00.000,131
9,2010-10-01T00:00:00.000,130


### Selecting all the rows and columns where the descriptor field = 'Street Flooding'

In [190]:
# Source domain for NYC Open Data on Socrata
socrata_domain = 'data.cityofnewyork.us'

# Data set id for NYC 311 on NYC Open Data on Socrata
socrata_dataset_identifier = 'erm2-nwe9'

# Socrata - The main class that interacts with the SODA API. We pass the source domain value
# of NYC Open data, the app token as 'None', and set the timeout parameter for '1,000 seconds'
client = Socrata(socrata_domain, None, timeout=1000)

# SoQL query string below:
# Select all columns where the descriptor is Street Flooding(SJ),
# sort the created date field in descending order and limit our records to 1,000.

query = """
SELECT 
    *
WHERE 
    descriptor == 'Street Flooding (SJ)'
ORDER BY 
    created_date DESC
LIMIT 
    1000
"""

# Requesting data from the NYC 311 data set
# and passing our query as a full SoQL query string
results = client.get(socrata_dataset_identifier, query=query)

# results is returned as JSON from API and converted to Python list of
# dictionaries by sodapy
print(type(results), 'Returned a list from our request.\n')

# Identifying type of first element of our results list
print(type(results[0]), 'However, request is actually a list of dictionaries.\n')

# Convert list of dictionaries to a pandas DataFrame
results_df = pd.DataFrame.from_records(results)

print(type(results_df), 'Convert list of dictionaries to DataFrame.')
print('Rows and columns of data:', results_df.shape)

# Writing out sample data as a csv
results_df.to_csv('sample_data_street_flooding.csv', index=False)

# Previewing the first five rows of our DataFrame
results_df.head()

<class 'list'> Returned a list from our request.

<class 'dict'> However, request is actually a list of dictionaries.

<class 'pandas.core.frame.DataFrame'> Convert list of dictionaries to DataFrame.
Rows and columns of data: (1000, 30)


,unique_key,created_date,agency,agency_name,complaint_type,descriptor,incident_zip,incident_address,street_name,cross_street_1,...,park_facility_name,park_borough,latitude,longitude,location,closed_date,resolution_description,resolution_action_updated_date,intersection_street_1,intersection_street_2
0,49630794,2021-01-26T20:25:00.000,DEP,Department of Environmental Protection,Sewer,Street Flooding (SJ),11237,340 WEIRFIELD STREET,WEIRFIELD STREET,IRVING AVE,...,Unspecified,BROOKLYN,40.695144569398586,-73.90712618152287,"{'latitude': '40.695144569398586', 'longitude'...",NaN,NaN,NaN,NaN,NaN
1,49632939,2021-01-26T17:41:00.000,DEP,Department of Environmental Protection,Sewer,Street Flooding (SJ),11378,59-34 58 ROAD,58 ROAD,GRAND AVE,...,Unspecified,QUEENS,40.720437393805675,-73.9068961273691,"{'latitude': '40.720437393805675', 'longitude'...",NaN,NaN,NaN,NaN,NaN
2,49618841,2021-01-25T18:09:00.000,DEP,Department of Environmental Protection,Sewer,Street Flooding (SJ),11432,185-23 80 ROAD,80 ROAD,CHEVY CHASE ST,...,Unspecified,QUEENS,40.7277798170635,-73.78278642955841,"{'latitude': '40.7277798170635', 'longitude': ...",NaN,NaN,NaN,NaN,NaN
3,49622225,2021-01-25T17:09:00.000,DEP,Department of Environmental Protection,Sewer,Street Flooding (SJ),NaN,PELHAM BAY PARK,PELHAM BAY PARK,BRUCKNER EXPRESSWAY EXIT 8 C NB,...,Unspecified,BRONX,NaN,NaN,NaN,2021-01-26T11:15:00.000,The Department of Environmental Protection inv...,2021-01-26T11:15:00.000,NaN,NaN
4,49610767,2021-01-24T19:20:00.000,DEP,Department of Environmental Protection,Sewer,Street Flooding (SJ),11420,124-19 HAWTREE CREEK ROAD,HAWTREE CREEK ROAD,124 ST,...,Unspecified,QUEENS,40.68086403950213,-73.81622814854843,"{'latitude': '40.68086403950213', 'longitude':...",2021-01-25T10:50:00.000,The Department of Environmental Protection inv...,2021-01-25T10:50:00.000,NaN,NaN


## Analyzing NYC 311 Data Sets with the Most Downloads

In [191]:
type(client)

sodapy.Socrata

In [192]:
type(client.datasets())

list

In [193]:
type(client.datasets()[0])

dict

In [194]:
len(client.datasets())

3143

In [195]:
# Reading in a list of dictionaries of our data into a pandas DataFrame
df = pd.DataFrame.from_records(client.datasets())

df.head()

,resource,classification,metadata,permalink,link,owner,creator,preview_image_url
0,"{'name': 'DOB Job Application Filings', 'id': ...","{'categories': ['economy', 'environment', 'hou...",{'domain': 'data.cityofnewyork.us'},https://data.cityofnewyork.us/d/ic3t-wcy2,https://data.cityofnewyork.us/Housing-Developm...,"{'id': '5fuc-pqz2', 'user_type': 'interactive'...","{'id': '5fuc-pqz2', 'user_type': 'interactive'...",NaN
1,"{'name': 'Civil Service List (Active)', 'id': ...","{'categories': [], 'tags': [], 'domain_categor...",{'domain': 'data.cityofnewyork.us'},https://data.cityofnewyork.us/d/vx8i-nprf,https://data.cityofnewyork.us/City-Government/...,"{'id': '5fuc-pqz2', 'user_type': 'interactive'...","{'id': '5fuc-pqz2', 'user_type': 'interactive'...",NaN
2,"{'name': 'TLC New Driver Application Status', ...","{'categories': ['transportation', 'environment...",{'domain': 'data.cityofnewyork.us'},https://data.cityofnewyork.us/d/dpec-ucu7,https://data.cityofnewyork.us/Transportation/T...,"{'id': '5fuc-pqz2', 'user_type': 'interactive'...","{'id': '5fuc-pqz2', 'user_type': 'interactive'...",NaN
3,"{'name': 'For Hire Vehicles (FHV) - Active', '...","{'categories': [], 'tags': [], 'domain_categor...",{'domain': 'data.cityofnewyork.us'},https://data.cityofnewyork.us/d/8wbx-tsch,https://data.cityofnewyork.us/Transportation/F...,"{'id': '5fuc-pqz2', 'user_type': 'interactive'...","{'id': '5fuc-pqz2', 'user_type': 'interactive'...",NaN
4,{'name': 'For Hire Vehicles (FHV) - Active Dri...,"{'categories': [], 'tags': [], 'domain_categor...",{'domain': 'data.cityofnewyork.us'},https://data.cityofnewyork.us/d/xjfq-wh2d,https://data.cityofnewyork.us/Transportation/F...,"{'id': '5fuc-pqz2', 'user_type': 'interactive'...","{'id': '5fuc-pqz2', 'user_type': 'interactive'...",NaN


In [196]:
# Only saving the dictionary in the resource column
df = df.resource

# Reading the dictionary in the resource column into a pandas DataFrame
df = pd.DataFrame.from_records(df)

df.head()

,name,id,parent_fxf,description,attribution,attribution_link,contact_email,type,updatedAt,createdAt,...,columns_field_name,columns_datatype,columns_description,columns_format,download_count,provenance,lens_view_type,blob_mime_type,hide_from_data_json,publication_date
0,DOB Job Application Filings,ic3t-wcy2,[],This dataset contains all job applications sub...,Department of Buildings (DOB),None,None,dataset,2021-01-27T21:18:28.000Z,2013-04-18T15:18:56.000Z,...,"[fuel_storage, fee_status, curb_cut, professio...","[Text, Text, Text, Text, Text, Text, Text, Tex...","[Fuel Storage Work Type? (X=Yes, Blank=No), T...","[{'align': 'right'}, {'align': 'right'}, {'ali...",36893.0,official,tabular,None,False,2020-06-22T18:23:35.000Z
1,Civil Service List (Active),vx8i-nprf,[],A Civil Service List consists of all candidate...,Department of Citywide Administrative Services...,None,None,dataset,2021-01-28T14:55:02.000Z,2016-06-14T21:12:15.000Z,...,"[exam_no, list_no, first_name, mi, last_name, ...","[text, number, text, text, text, number, text,...",[A four (4) digit number that identifies a civ...,"[{'displayStyle': 'plain', 'align': 'left'}, {...",34291.0,official,tabular,None,False,2021-01-28T14:55:02.000Z
2,TLC New Driver Application Status,dpec-ucu7,[],THIS DATASET IS UPDATED SEVERAL TIMES PER DAY....,Taxi and Limousine Commission (TLC),None,None,dataset,2021-01-28T17:16:32.000Z,2016-05-17T18:43:43.000Z,...,"[status, driver_exam, app_no, app_date, fru_in...","[Text, Text, Number, Calendar date, Text, Text...","[""Incomplete"": Your application is missing req...","[{'displayStyle': 'plain', 'align': 'left'}, {...",35363.0,official,tabular,None,False,2019-12-17T18:44:57.000Z
3,For Hire Vehicles (FHV) - Active,8wbx-tsch,[],"<b>PLEASE NOTE:</b> This dataset, which includ...",Taxi and Limousine Commission (TLC),None,None,dataset,2021-01-27T20:31:43.000Z,2015-07-16T17:33:32.000Z,...,"[active, vehicle_license_number, name, license...","[text, text, text, text, calendar_date, text, ...","[Permit active or not\n, FHV Vehicle License N...","[{'displayStyle': 'plain', 'align': 'left'}, {...",254346.0,official,tabular,None,False,2021-01-27T20:31:43.000Z
4,For Hire Vehicles (FHV) - Active Drivers,xjfq-wh2d,[],"<b>PLEASE NOTE:</b> This dataset, which includ...",Taxi and Limousine Commission (TLC),None,None,dataset,2021-01-27T20:20:02.000Z,2015-07-16T17:24:02.000Z,...,"[license_number, name, type, expiration_date, ...","[number, text, text, calendar_date, text, cale...","[FHV License Number\n, Driver Name\n\n, Type o...","[{'precisionStyle': 'standard', 'noCommas': 't...",222191.0,official,tabular,None,False,2021-01-27T20:20:02.000Z


In [197]:
len(df)

3143

In [198]:
# Sorting the data sets by download_count
df[['name', 'download_count']].sort_values(by='download_count', ascending=False).head()

,name,download_count
33,Demographic Statistics By Zip Code,1014101.0
1237,Overhead Electronic Signs,429382.0
5,311 Service Requests from 2010 to Present,398304.0
10,Medallion Drivers - Active,287979.0
3,For Hire Vehicles (FHV) - Active,254346.0


In [199]:
highest_downloaded = df[['name', 'download_count']].sort_values(by='download_count', ascending=False)

print('The data set {}'.format(highest_downloaded['name'].iloc[0]), \
     'has {} downloads'.format(f"{highest_downloaded['download_count'].iloc[0]:,.0f}"), \
     'and is the most downloaded data set on NYC Open Data.')

The data set Demographic Statistics By Zip Code has 1,014,101 downloads and is the most downloaded data set on NYC Open Data.


# Retrieving Data Directly from SODA API

In [200]:
# Source domain for NYC Open Data on Socrata
socrata_domain = 'data.cityofnewyork.us'

# Data set id for NYC 311 on NYC Open Data on Socrata
socrata_dataset_identifier = 'erm2-nwe9'

url = 'https://' + socrata_domain + '/resource/' + socrata_dataset_identifier + '.csv?$limit=20'
print(url)

df = pd.read_csv(url)
print(df.shape)

df.head()

https://data.cityofnewyork.us/resource/erm2-nwe9.csv?$limit=20
(20, 41)


,unique_key,created_date,closed_date,agency,agency_name,complaint_type,descriptor,location_type,incident_zip,incident_address,...,vehicle_type,taxi_company_borough,taxi_pick_up_location,bridge_highway_name,bridge_highway_direction,road_ramp,bridge_highway_segment,latitude,longitude,location
0,49633221,2021-01-27T01:59:12.000,NaN,DOHMH,Department of Health and Mental Hygiene,Rodent,Rat Sighting,1-2 Family Dwelling,11238,465 SAINT JOHN'S PLACE,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,40.673467,-73.962338,"\n, \n(40.6734674790127, -73.9623383832787)"
1,49628739,2021-01-27T01:59:02.000,NaN,NYPD,New York City Police Department,Illegal Parking,Blocked Hydrant,Street/Sidewalk,11365,67-10 190 LANE,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,40.736984,-73.781609,"\n, \n(40.73698352494025, -73.78160893379135)"
2,49627727,2021-01-27T01:57:54.000,NaN,NYPD,New York City Police Department,Noise - Residential,Banging/Pounding,Residential Building/House,11101,40-03 10 STREET,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,40.756670,-73.944561,"\n, \n(40.75667046806001, -73.9445613744302)"
3,49630814,2021-01-27T01:55:27.000,NaN,NYPD,New York City Police Department,Noise - Residential,Loud Music/Party,Residential Building/House,11365,67-10 190 LANE,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,40.736984,-73.781609,"\n, \n(40.73698352494025, -73.78160893379135)"
4,49626583,2021-01-27T01:54:11.000,NaN,NYPD,New York City Police Department,Abandoned Vehicle,With License Plate,Street/Sidewalk,11434,115 AVENUE,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,40.690441,-73.776349,"\n, \n(40.690441338844835, -73.77634885043103)"


In [201]:
# Source domain for NYC Open Data on Socrata
socrata_domain = 'data.cityofnewyork.us'

# Data set id for NYC 311 on NYC Open Data on Socrata
socrata_dataset_identifier = 'erm2-nwe9'

year = '2020'
column = 'created_date'

url = 'https://data.cityofnewyork.us/resource/erm2-nwe9.csv?' \
'$where=' + column + '%20>=%20%27' + year + '%27'  \
'&$limit=20'
print(url)

df = pd.read_csv(url)
print(df.shape)

df.head()

https://data.cityofnewyork.us/resource/erm2-nwe9.csv?$where=created_date%20>=%20%272020%27&$limit=20
(20, 41)


,unique_key,created_date,closed_date,agency,agency_name,complaint_type,descriptor,location_type,incident_zip,incident_address,...,vehicle_type,taxi_company_borough,taxi_pick_up_location,bridge_highway_name,bridge_highway_direction,road_ramp,bridge_highway_segment,latitude,longitude,location
0,45285347,2020-01-01T00:00:00.000,2020-01-10T00:00:01.000,DOHMH,Department of Health and Mental Hygiene,Food Poisoning,3 or More,Restaurant/Bar/Deli/Bakery,11229,3442 NOSTRAND AVENUE,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,40.600129,-73.941843,"\n, \n(40.6001292057807, -73.94184291675883)"
1,45285651,2020-01-01T00:00:00.000,2020-01-02T00:00:01.000,DOHMH,Department of Health and Mental Hygiene,Food Poisoning,1 or 2,Restaurant/Bar/Deli/Bakery,10458,2701 DECATUR AVENUE,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,40.864866,-73.888783,"\n, \n(40.86486556770799, -73.88878325729915)"
2,45285821,2020-01-01T00:00:00.000,2020-01-02T09:50:09.000,DOHMH,Department of Health and Mental Hygiene,Food Poisoning,1 or 2,Other (Explain Below),11203,5707 CHURCH AVENUE,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,40.652536,-73.923540,"\n, \n(40.65253575905768, -73.92353994017134)"
3,45287907,2020-01-01T00:00:00.000,2020-01-02T00:00:01.000,DOHMH,Department of Health and Mental Hygiene,Food Poisoning,3 or More,Restaurant/Bar/Deli/Bakery,11214,1602 SHORE PARKWAY,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,40.595653,-74.000173,"\n, \n(40.59565343138651, -74.00017283917487)"
4,45288120,2020-01-01T00:00:00.000,2020-01-02T09:51:29.000,DOHMH,Department of Health and Mental Hygiene,Food Poisoning,3 or More,Restaurant/Bar/Deli/Bakery,10455,748 EAST 149 STREET,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,40.812996,-73.907973,"\n, \n(40.81299645614164, -73.90797324533352)"


In [202]:
url = 'https://data.cityofnewyork.us/resource/erm2-nwe9.csv?$query=SELECT%20*%20LIMIT%2020'
print(url)

df = pd.read_csv(url)
print(df.shape)

df.head()

https://data.cityofnewyork.us/resource/erm2-nwe9.csv?$query=SELECT%20*%20LIMIT%2020
(20, 41)


,unique_key,created_date,closed_date,agency,agency_name,complaint_type,descriptor,location_type,incident_zip,incident_address,...,vehicle_type,taxi_company_borough,taxi_pick_up_location,bridge_highway_name,bridge_highway_direction,road_ramp,bridge_highway_segment,latitude,longitude,location
0,49633221,2021-01-27T01:59:12.000,NaN,DOHMH,Department of Health and Mental Hygiene,Rodent,Rat Sighting,1-2 Family Dwelling,11238,465 SAINT JOHN'S PLACE,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,40.673467,-73.962338,"\n, \n(40.6734674790127, -73.9623383832787)"
1,49628739,2021-01-27T01:59:02.000,NaN,NYPD,New York City Police Department,Illegal Parking,Blocked Hydrant,Street/Sidewalk,11365,67-10 190 LANE,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,40.736984,-73.781609,"\n, \n(40.73698352494025, -73.78160893379135)"
2,49627727,2021-01-27T01:57:54.000,NaN,NYPD,New York City Police Department,Noise - Residential,Banging/Pounding,Residential Building/House,11101,40-03 10 STREET,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,40.756670,-73.944561,"\n, \n(40.75667046806001, -73.9445613744302)"
3,49630814,2021-01-27T01:55:27.000,NaN,NYPD,New York City Police Department,Noise - Residential,Loud Music/Party,Residential Building/House,11365,67-10 190 LANE,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,40.736984,-73.781609,"\n, \n(40.73698352494025, -73.78160893379135)"
4,49626583,2021-01-27T01:54:11.000,NaN,NYPD,New York City Police Department,Abandoned Vehicle,With License Plate,Street/Sidewalk,11434,115 AVENUE,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,40.690441,-73.776349,"\n, \n(40.690441338844835, -73.77634885043103)"


In [203]:
query = """
        SELECT %20
            * %20
        WHERE %20
            created_date %20 >= %20%27 2020 %27%20 
        LIMIT %20
            20
        """

query = ''.join(query.split())
print('query:', query)

url = 'https://data.cityofnewyork.us/resource/erm2-nwe9.csv?$query=' + query
print('url:', url)

df = pd.read_csv(url)
print(df.shape)

df.head()

query: SELECT%20*%20WHERE%20created_date%20>=%20%272020%27%20LIMIT%2020
url: https://data.cityofnewyork.us/resource/erm2-nwe9.csv?$query=SELECT%20*%20WHERE%20created_date%20>=%20%272020%27%20LIMIT%2020
(20, 41)


,unique_key,created_date,closed_date,agency,agency_name,complaint_type,descriptor,location_type,incident_zip,incident_address,...,vehicle_type,taxi_company_borough,taxi_pick_up_location,bridge_highway_name,bridge_highway_direction,road_ramp,bridge_highway_segment,latitude,longitude,location
0,45285347,2020-01-01T00:00:00.000,2020-01-10T00:00:01.000,DOHMH,Department of Health and Mental Hygiene,Food Poisoning,3 or More,Restaurant/Bar/Deli/Bakery,11229,3442 NOSTRAND AVENUE,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,40.600129,-73.941843,"\n, \n(40.6001292057807, -73.94184291675883)"
1,45285651,2020-01-01T00:00:00.000,2020-01-02T00:00:01.000,DOHMH,Department of Health and Mental Hygiene,Food Poisoning,1 or 2,Restaurant/Bar/Deli/Bakery,10458,2701 DECATUR AVENUE,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,40.864866,-73.888783,"\n, \n(40.86486556770799, -73.88878325729915)"
2,45285821,2020-01-01T00:00:00.000,2020-01-02T09:50:09.000,DOHMH,Department of Health and Mental Hygiene,Food Poisoning,1 or 2,Other (Explain Below),11203,5707 CHURCH AVENUE,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,40.652536,-73.923540,"\n, \n(40.65253575905768, -73.92353994017134)"
3,45287907,2020-01-01T00:00:00.000,2020-01-02T00:00:01.000,DOHMH,Department of Health and Mental Hygiene,Food Poisoning,3 or More,Restaurant/Bar/Deli/Bakery,11214,1602 SHORE PARKWAY,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,40.595653,-74.000173,"\n, \n(40.59565343138651, -74.00017283917487)"
4,45288120,2020-01-01T00:00:00.000,2020-01-02T09:51:29.000,DOHMH,Department of Health and Mental Hygiene,Food Poisoning,3 or More,Restaurant/Bar/Deli/Bakery,10455,748 EAST 149 STREET,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,40.812996,-73.907973,"\n, \n(40.81299645614164, -73.90797324533352)"
